In [2]:
import pandas as pd

In [12]:
df=pd.read_csv('../data/scraped/thegioididong_phones_2025-09-01.csv')

df['updated_at']=pd.to_datetime(df['updated_at'])

df.drop_duplicates(subset='id',keep='last').sort_values(by='id')
# df.sort_values(by='id')

,id,name,price,brand,category,chipset,ram,storage,screen_size,screen_tech,screen_resolution,screen_frequency,operating_system,battery,rating,reviews_count,url,created_at,updated_at
661,207956,Điện thoại Nokia 220 4G,980000,Nokia,Phone,Unisoc T107 8 nhân,64 MB,128 MB,"2.8""",NaN,QVGA (240 x 320 Pixels),NaN,S30+,1450 mAh,4.9,61302.0,https://www.thegioididong.com/dtdd/nokia-220-4g,06/2024,2025-09-01 23:34:43
509,223602,Điện thoại iPhone 13 128GB,11690000,iPhone (Apple),Smartphone,Apple A15 Bionic,4 GB,128 GB,"6.1""",OLED,Super Retina XDR (1170 x 2532 Pixels),60 Hz,iOS 17,3240 mAh,4.9,243243.0,https://www.thegioididong.com/dtdd/iphone-13,09/2021,2025-09-01 23:34:12
618,240259,Điện thoại iPhone 14 128GB,13190000,iPhone (Apple),Smartphone,Apple A15 Bionic,6 GB,128 GB,"6.1""",OLED,Super Retina XDR (1170 x 2532 Pixels),60 Hz,iOS 17,3279 mAh,4.9,52853.0,https://www.thegioididong.com/dtdd/iphone-14,09/2022,2025-09-01 23:34:34
502,265311,Điện thoại Masstel IZI 10,410000,Masstel,Phone,Unisoc UMS9117,48 MB,128 MB,"1.77""",TFT LCD,QQVGA (128 x 160 Pixels),NaN,NaN,1000 mAh,4.9,286066.0,https://www.thegioididong.com/dtdd/masstel-izi-10,12/2021,2025-09-01 23:34:12
791,281570,Điện thoại iPhone 15 128GB,15590000,iPhone (Apple),Smartphone,Apple A16 Bionic,6 GB,128 GB,"6.1""",OLED,Super Retina XDR (1179 x 2556 Pixels),60 Hz,iOS 17,3349 mAh,4.9,91248.0,https://www.thegioididong.com/dtdd/iphone-15,09/2023,2025-09-01 23:35:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,341690,Điện thoại Samsung Galaxy A17 5G 8GB/256GB,7090000,Samsung,Smartphone,Exynos 1330,8 GB,256 GB,"6.7""",Super AMOLED,Full HD+ (1080 x 2340 Pixels),90 Hz,Android 15,5000 mAh,5.0,1379.0,https://www.thegioididong.com/dtdd/samsung-gal...,08/2025,2025-09-01 23:34:12
357,341802,Điện thoại Samsung Galaxy A07 4GB/64GB,3090000,Samsung,Smartphone,MediaTek Helio G99,4 GB,64 GB,"6.7""",IPS LCD,HD+ (720 x 1600 Pixels),90 Hz,Android 15,5000 mAh,NaN,NaN,https://www.thegioididong.com/dtdd/samsung-gal...,08/2025,2025-09-01 23:33:42
93,341837,Điện thoại Tecno Spark Go 2 3GB/64GB,2190000,Tecno,Smartphone,Unisoc T7250 8 nhân,3 GB,64 GB,"6.67""",IPS LCD,HD+ (720 x 1600 Pixels),120 Hz,Android 15 Go,5000 mAh,NaN,NaN,https://www.thegioididong.com/dtdd/tecno-spark...,08/2025,2025-09-01 23:32:55
768,342406,Điện thoại OPPO A5i 6GB/128GB,4490000,OPPO,Smartphone,Snapdragon 6s Gen 1 8 nhân,6 GB,128 GB,"6.67""",IPS LCD,HD+ (720 x 1604 Pixels),90 Hz,Android 14,5100 mAh,5.0,5.0,https://www.thegioididong.com/dtdd/oppo-a5i-6g...,07/2025,2025-09-01 23:35:03
